In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
EMBEDDING_FILE = 'adfasfad'#'/home/kai/data/resources/glove/glove.840B.300d.txt' #glove.twitter.27B.200d.txt
PATH = '~/data/toxic/data/'
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'test.csv')
#train = pd.read_csv(PATH + 'cleaned_train.csv')
#test = pd.read_csv(PATH + 'cleaned_test.csv')

In [3]:
comment_col = 'comment_text' # 'comment_text_cleaned'

In [4]:
X_train = train[comment_col].str.lower()
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

X_test = test[comment_col].str.lower()

In [5]:
max_features=100000
maxlen=150
embed_size=300
if 'twitter' in EMBEDDING_FILE:
    embed_size = 200

In [6]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [7]:
tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(X_train)+list(X_test))
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)

In [8]:
# import gensim
# file = '/home/kai/data/resources/google/freebase-vectors-skipgram1000-en.bin'#GoogleNews-vectors-negative300.bin'
# w2v_model = gensim.models.KeyedVectors.load_word2vec_format(file, binary=True)  

In [9]:
# embeddings_index = {}
# with open(EMBEDDING_FILE,encoding='utf8') as f:
#     for line in f:
#         values = line.rstrip().rsplit(' ')
#         word = values[0]
#         coefs = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = coefs

In [24]:
import numpy as np
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32') 
#file = '/home/kai/data/resources/lexvec/lexvec.commoncrawl.300d.W.pos.vectors'
file = '/home/kai/data/resources/dependency/bow5.words'#deps.words' # don't include .bz2

embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(file, encoding='utf8'))
#embeddings_index.pop('2000000')  # only needed when using lexvec

In [25]:
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
not_found_count = 0
for word, i in word_index.items():
    if i >= max_features:
        continue
    try: 
        embedding_vector = embeddings_index.get(word) # w2v_model['/en/'+ word] #w2v_model[word]#
    except KeyError:
        embedding_vector = None #np.zeros(embed_size)
        not_found_count += 1
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print(not_found_count)

0


In [26]:
embedding_matrix.shape

(100000, 300)

In [15]:
sequence_input = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x) 
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.1)(x)
preds = Dense(6, activation="sigmoid")(x)
model = Model(sequence_input, preds)

# filepath="lex_300d_weights_b512_ep6.hdf5"
# print('load model: ' + str(filepath))
# model.load_weights(filepath)

model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

In [16]:
batch_size = 128
epochs = 6
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state=233)

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [17]:
filepath= "dep_300d_weights_b128_ep6.hdf5" #"twit_weights_base.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
ra_val = RocAucEvaluation(validation_data=(X_val, y_val), interval = 1)
callbacks_list = [ra_val,checkpoint, early]

In [18]:
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1)
#Loading model weights
model.load_weights(filepath)
print('Predicting using the best model/epoch so far....')
y_pred = model.predict(x_test,batch_size=1024,verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/6
143488/143613 [============================>.] - ETA: 0s - loss: 0.0745 - acc: 0.9761
 ROC-AUC - epoch: 1 - score: 0.970835

Epoch 00001: val_acc improved from -inf to 0.97814, saving model to dep_300d_weights_b128_ep6.hdf5
143613/143613 [==============================] - 316s 2ms/step - loss: 0.0745 - acc: 0.9761 - val_loss: 0.0635 - val_acc: 0.9781
Epoch 2/6
143488/143613 [============================>.] - ETA: 0s - loss: 0.0545 - acc: 0.9806
 ROC-AUC - epoch: 2 - score: 0.979198

Epoch 00002: val_acc improved from 0.97814 to 0.98171, saving model to dep_300d_weights_b128_ep6.hdf5
143613/143613 [==============================] - 317s 2ms/step - loss: 0.0545 - acc: 0.9806 - val_loss: 0.0511 - val_acc: 0.9817
Epoch 3/6
143488/143613 [============================>.] - ETA: 0s - loss: 0.0510 - acc: 0.9813
 ROC-AUC - epoch: 3 - score: 0.982574

Epoch 00003: val_acc did not improve
143613/143613 [==============================] -

In [21]:
y_pred.shape

(153164, 6)

In [20]:
submission = pd.read_csv(PATH + 'sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
import time
sub_id = int(time.time())
print(sub_id)
submission.to_csv('./BaseEstPreds/' + 'rnn_w_conv_dep_d300_b128_ep6' + str(sub_id) + '.csv', index=False)

1520614488


In [70]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 150, 300)     30000000    input_10[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_9 (SpatialDro (None, 150, 300)     0           embedding_10[0][0]               
__________________________________________________________________________________________________
bidirectional_10 (Bidirectional (None, 150, 256)     329472      spatial_dropout1d_9[0][0]        
__________________________________________________________________________________________________
conv1d_10 

In [43]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 150, 300)     30000000    input_5[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_5 (SpatialDro (None, 150, 300)     0           embedding_5[0][0]                
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 150, 256)     329472      spatial_dropout1d_5[0][0]        
__________________________________________________________________________________________________
conv1d_5 (